## Test Scrape Notebook

In [63]:
## Import library as establish paths
from pypdf import PdfReader
import translators as ts
import pandas as pd
import nltk
from docx import Document


headername = r"Reinier_OldenBarnevelt"

# Reinier_oldenbarnevelt is a jpg. more processing to do here

filename = 'Reinier_oldenbarnevelt'

first_page = 1
last_page = 1

source_jpg_input_path = r'./input/' + filename + r'.jpg'
source_pdf_input_path = r'./input/' + filename + r'.pdf'
source_text_output_path = r'./output/' + filename + '_nntlLTSM.txt'
trans_table_output_path = r'./output/' + filename + '_translation_nntlLTSM.csv'
trans_text_output_path = r'./output/' + filename + '_translated_nntlLTSM.txt'
trans_doc_output_path = r'./output/' + filename + '_translated_nntlLTSM.docx'


## Renier's JPG

In [39]:
## for Renier's jpeg
# text recognition
import cv2
import pytesseract

# read image
if filename == 'Reinier_oldenbarnevelt':
    print('jpeg workflow engaged!')
    img = cv2.imread(source_jpg_input_path)



jpeg workflow engaged!


In [3]:
## config tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [27]:
## optional: gauss the image first
# img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
# img = cv2.GaussianBlur(img, (5, 5), 0)

In [64]:
if filename == 'Reinier_oldenbarnevelt':
    # configurations
    # config = ('-l deu_latf --oem 1 --psm 1')
    # config = ('-l frk --oem 1 --psm 1')
    config = ('-l ntl --oem 1 --psm 1')
    print("scraping text from jpg...")
    text = pytesseract.image_to_string(img, config=config)

    # print text
    text = text.split()

 
    print(text)

scraping text from jpg...


In [53]:
if filename == 'Reinier_oldenbarnevelt':
    # Write out to raw text
    long_text = " ".join(text)

    with open(source_text_output_path, 'w', encoding="utf-8") as f:
        f.write(long_text)

## Data Prep

In [ ]:
if filename != 'Reinier_oldenbarnevelt':
    reader = PdfReader(source_pdf_input_path)
    number_of_pages = len(reader.pages)
    page = reader.pages[0]
    text = page.extract_text()

In [ ]:
if filename != 'Reinier_oldenbarnevelt':
    print(number_of_pages - 856)
# Pulling pages 775 to 830

In [ ]:
if filename != 'Reinier_oldenbarnevelt':
    # Trying to extract the range into a text file
    select_range = list(range(first_page,last_page,1))

    long_text = ''

    for p in select_range:
        page = reader.pages[p]
        text = page.extract_text()
        long_text = long_text + text
        
    print(len(long_text))

In [ ]:
if filename != 'Reinier_oldenbarnevelt':
    # Write out to raw text
    with open(source_text_output_path, 'w', encoding="utf-8") as f:
        f.write(long_text)

## Translation

In [54]:
# read raw text
with open(source_text_output_path, 'r', encoding="utf-8") as f:
    long_text = f.read()
print(len(long_text))


5107


In [55]:
## step 1: convert raw text into sentences in a df
from nltk.tokenize import sent_tokenize

sentences_list = sent_tokenize(long_text)

print(len(sentences_list))



24


In [56]:
## establish a dataframe to store input and output
trans_df = pd.DataFrame()
trans_df['raw_nl'] = sentences_list

print(trans_df.head())

                                              raw_nl
0   k STIERT AAENTAITIGET EI REDERTIN NWOORDERS DiE.
1  GENSTYCERT 11 DER DRIiE, SRS ann | LIESE ek KA...
2  NE DN <E WUaerachtichverhael/ vantghenein de E...
3  wi Lſgotat Welſtant vaiteen goede Bepu- „CA bl...
4                             metgröate reverentien.


In [57]:
# import the library
import googletrans
from googletrans import Translator

In [58]:
# create a translator object
translator = Translator()

# use translate method to translate a string - by default, the destination language is english
translated = translator.translate(trans_df['raw_nl'][0],
                                  src='nl',
                                  dest= 'en')

# the translate method returns an object
print(translated.text)

k BULLET AAENTAITIGET EI REDERTIN NWOORDERS Die.


In [59]:
## df style

trans_df['trans_en'] = trans_df['raw_nl'].apply(lambda x: translator.translate(x,
                                  src='nl',
                                  dest= 'en').text)
print(trans_df.head())

                                              raw_nl  \
0   k STIERT AAENTAITIGET EI REDERTIN NWOORDERS DiE.   
1  GENSTYCERT 11 DER DRIiE, SRS ann | LIESE ek KA...   
2  NE DN <E WUaerachtichverhael/ vantghenein de E...   
3  wi Lſgotat Welſtant vaiteen goede Bepu- „CA bl...   
4                             metgröate reverentien.   

                                            trans_en  
0   k BULLET AAENTAITIGET EI REDERTIN NWOORDERS Die.  
1  GENSTYCERT 11 DER DRIiE, SRS ann | LIESE ek KA...  
2  NE DN <E WUaerachtichverhael/ vantghenein the ...  
3  wi Lſgotat Welſtant vaiteen good Bepu- „CA bli...  
4                             with size reverentien.  


In [60]:
## write out the table
# trans_df.to_csv(trans_table_output_path)

In [61]:
## write out the translated text
long_text_translated = ''.join(trans_df["trans_en"])
# Write out to raw text
# with open(trans_text_output_path, 'w', encoding="utf-8") as f:
    # f.write(long_text_translated)


In [62]:
## as word doc
document = Document()
document.add_heading(headername, level=1)
document.add_paragraph(long_text_translated)

document.save(trans_doc_output_path)